In [1]:
import sys
import pandas as pd
import numpy as np
import datetime
import glob
import gc
import os

#========================================================================
# Args
#========================================================================
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month']

win_path = f'../features/4_winner/*.gz'
fname=''
# submit = pd.read_csv('../input/sample_submission.csv')
submit = []

HOME = os.path.expanduser('~')

sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from preprocessing import get_ordinal_mapping
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

2019-01-03 10:28:52,034 utils 366 [INFO]    [logger_func] start 


In [11]:
#========================================================================
# Data Load
base = utils.read_df_pkl('../input/base*')
win_path_list = glob.glob(win_path)
train_path_list = []
test_path_list = []
for path in win_path_list:
    if path.count('train'):
        train_path_list.append(path)
    elif path.count('test'):
        test_path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
train_feature_list = utils.parallel_load_data(path_list=train_path_list)
test_feature_list = utils.parallel_load_data(path_list=test_path_list)
train = pd.concat(train_feature_list, axis=1)
train = pd.concat([base_train, train], axis=1)
test = pd.concat(test_feature_list, axis=1)
test = pd.concat([base_test, test], axis=1)

train_id = train[key].values
test_id = test[key].values

y = train[[key, target]]
# y[target] = y[target].map(lambda x: 1 if x<-30 else 0)
train.drop(target, axis=1, inplace=True)

for col in train.columns:
    if len(train[train[col].isnull()])==0 or len(test[test[col].isnull()])==0:
        continue
        
    imp_train = train[col].median()
    imp_test = test[col].median()
    
    train[col].fillna(imp_train, inplace=True)
    test[col].fillna(imp_test, inplace=True)

print(train.shape)
# FFMは最後の列がラベルになる
train.sort_index(axis=1, inplace=True)
test.sort_index(axis=1, inplace=True)
train = train.merge(y, how='inner', on=key)

100%|██████████| 3/3 [00:00<00:00, 85.05it/s]


(201917, 19)


In [12]:
train.head()

,117_ah1_auth1_hist_card_id_size@,117_ah1_auth1_hist_category_1_mean@,117_ah1_auth1_hist_category_1_sum@,117_ah1_auth1_hist_category_2_mean_mean@,117_ah1_auth1_hist_category_3_mean_mean@,117_ah1_auth1_hist_month_diff_mean@,117_ah1_auth1_hist_purchase_amount_max@,117_ah1_auth1_hist_purchase_amount_mean@,117_ah1_auth1_hist_purchase_amount_sum@,118_ah0_auth0_hist_card_id_size@,118_ah0_auth0_hist_category_1_mean@,118_ah0_auth0_hist_category_1_sum@,118_ah0_auth0_hist_category_2_mean_mean@,118_ah0_auth0_hist_category_3_mean_mean@,118_ah0_auth0_hist_month_diff_mean@,118_ah0_auth0_hist_purchase_amount_max@,118_ah0_auth0_hist_purchase_amount_mean@,118_ah0_auth0_hist_purchase_amount_sum@,card_id,target
0,247,0.000000,0,-0.605846,-0.641168,10.165992,2.258395,-0.637235,-157.397016,13.0,0.000000,0.0,9.817255,9.698610,10.153846,-0.431922,-0.659363,-8.571723,C_ID_92a2005557,-0.820283
1,339,0.085546,29,-0.606068,-0.495468,11.165192,4.630299,-0.616175,-208.883450,11.0,0.181818,2.0,9.817255,1.043947,11.272727,1.942838,-0.102081,-1.122886,C_ID_3d0044924f,0.392913
2,41,0.000000,0,-0.589559,-0.641168,10.097561,-0.145847,-0.678742,-27.828424,2.0,0.000000,0.0,6.764233,13.296816,10.000000,-0.637515,-0.669484,-1.338967,C_ID_d639edf6cd,0.688056
3,77,0.155844,12,-0.594682,-0.554738,10.116883,1.445596,-0.642745,-49.491364,6.0,0.000000,0.0,9.817255,4.712152,10.200000,-0.499331,-0.633717,-2.714321,C_ID_186d6a6901,0.142495
4,128,0.093750,12,-0.593270,-0.591703,10.101562,6.992617,-0.539379,-69.040465,5.0,0.600000,3.0,5.393454,1.075873,10.000000,7.193041,4.070562,20.352809,C_ID_cdbd2c0db2,-0.159749


In [15]:
%load_ext autoreload
%autoreload 2
from xlearn_test import convert_to_ffm

cat_cols = utils.get_categorical_features(df=train, ignore_list=ignore_list)
num_cols = utils.get_numeric_features(df=train, ignore_list=ignore_list)

convert_to_ffm(df=train, numerics=num_cols, categories=cat_cols, features=num_cols+cat_cols, fname='0103_check')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from sklearn.model_selection import StratifiedKFold
seed = 1208

train['outliers'] = train[target].map(lambda x: 1 if x<-30 else 0)
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
outliers = train['outliers'].values
train.drop('outliers', axis=1, inplace=True)
kfold = folds.split(train, outliers)

train[target] =  train[target].map(lambda x: 1 if x<-30 else 0)
y = train[target]

use_cols = [col for col in train.columns if col not in ignore_list]

for n_fold, (trn_idx, val_idx) in enumerate(kfold):
    x_train, y_train = train[use_cols].iloc[trn_idx, :].values, y.iloc[trn_idx].values
    x_val, y_val = train[use_cols].iloc[val_idx, :].values, y.iloc[val_idx].values
    
    y_train = y_train.astype('int8')
    y_val = y_val.astype('int8')
    
    tmp = np.hstack((x_train, y_train.reshape(len(y_train), 1)))
    tmp = pd.DataFrame(tmp)
    tmp.to_csv(f'../input/ffm_train_{n_fold}.csv', index=False, header=False)
    del tmp
    gc.collect()
    
    tmp = np.hstack((x_val, y_val.reshape(len(y_val), 1)))
    tmp = pd.DataFrame(tmp)
    tmp.to_csv(f'../input/ffm_val_{n_fold}.csv', index=False, header=False)
    del tmp
    gc.collect()
    sys.exit()
    
train.to_csv('../input/ffm_train.csv', index=False)
test.to_csv('../input/ffm_test.csv', index=False)